# Ultima alteração 28.10.2020

In [5]:
def get_properties(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    parameters = {'S_eV':{'expression':'Entropy','after':0,'field':2},
                  'S_J/molK':{'expression':'Entropy','after':1,'field':1},
                  'G_eV':{'expression':'Total free energy :','after':1,'field':4},
                  'G_kJ/molK':{'expression':'Total free energy (in kJmol-1)','after':1,'field':4},
                  **{f'sdlc{i}{j}':{'expression':'Static dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'hfdlc{i}{j}':{'expression':'High frequency dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'C{i}{j}':{'expression':'Elastic Constant Matrix: (Units=GPa)','after':4+i,'field':j} for i in (1,2,3,4,5,6) for j in (1,2,3,4,5,6)},
                  **{f'bulk_modulus_{i}':{'expression':'Mechanical properties :','after':5,'field':4+('Reuss', 'Voigt', 'Hill').index(i)} for i in ('Reuss', 'Voigt', 'Hill')},
                  'lattice_parameter_a':{'expression':'Comparison of initial and final structures :','after':6,'field':2},
                  'lattice_parameter_b':{'expression':'Comparison of initial and final structures :','after':7,'field':2},
                  'lattice_parameter_c':{'expression':'Comparison of initial and final structures :','after':8,'field':2}}

    for element in wanted_properties:
        expression = parameters[element]['expression']
        after = parameters[element]['after']
        field = parameters[element]['field']  
        for line_number,line in enumerate(lines_bottom):
            if expression in line: 
                properties[wanted_properties[wanted_properties.index(element)]] = lines_bottom[line_number+after].split()[field]

    return(properties)

print(get_properties("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out",['lattice_parameter_a','lattice_parameter_b','lattice_parameter_c']))

{'lattice_parameter_a': 'a5.710467', 'lattice_parameter_b': 'b5.710467', 'lattice_parameter_c': 'c5.710467'}
